In [1]:
import findspark
findspark.init() 
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr').getOrCreate()

In [30]:
from pyspark.sql.functions import countDistinct
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [4]:
cruise_ship = spark.read.csv('file:///home/manohar_l/Manohar_Personal_Projects/cruise_ship_info.csv', inferSchema=True,header=True)

In [10]:
cruise_ship.printSchema()
cruise_ship.describe().show()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|       

In [41]:
cruise_ship.agg(countDistinct('passengers').alias('c')).collect()

[Row(c=104)]

In [7]:
cassembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density'],
                            outputCol='features')

In [8]:
cruise = cassembler.transform(cruise_ship)

In [9]:
cruise.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [11]:
cruise_final = cruise.select('features','crew')

In [12]:
cruise_final.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [42]:
train_data, test_data = cruise_final.randomSplit([0.7,0.3])

In [43]:
train_data.describe().show()
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              106|
|   mean|8.112735849056614|
| stddev|3.449218139981699|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               52|
|   mean|7.144807692307693|
| stddev|3.556830760709852|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [44]:
lr = LinearRegression(labelCol='crew', featuresCol='features')

In [45]:
lr_cruise = lr.fit(train_data)

In [46]:
cruise_test = lr_cruise.evaluate(test_data)

In [47]:
cruise_test.rootMeanSquaredError

1.1881046080419844

In [48]:
cruise_test.r2

0.8862330472881746

In [54]:
cruise_ship.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+

